In [6]:
import os
from dotenv import load_dotenv
load_dotenv()  # 讀取 .env 檔案
api_key = os.getenv("GOOGLE_API_KEY")
Mongo_url = os.getenv("MONGO_URI")
Database_Name = os.getenv("Database_Name")
Collection_Name = os.getenv("Collection_Name")

In [7]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "eveng": {
            "transport": "streamable_http",
            "url": "http://127.0.0.1:4200/my-custom-path/"
        },
    }
)

tools = await client.get_tools()

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model= "gemini-2.5-flash-preview-05-20",
    temperature=1.0,
    max_retries=2,
    google_api_key=api_key,
)

model = llm.bind_tools(tools)

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.mongodb.aio import AsyncMongoDBSaver
import time

async with AsyncMongoDBSaver.from_conn_string(Mongo_url) as checkpointer:
	
	agent = create_react_agent(
		model = model,
		tools = tools,
		checkpointer = checkpointer,
		prompt="""當用戶請你幫忙解析影片時，請使用download(MCP tool)下載影片，
				當影片超過300秒(5分鐘)時使用audio_cut(MCP tool)依每5分鐘的長度分割成多個片段，
				再使用whisper_tool(MCP tool)產生字幕檔並根據回傳的文字進行影片內容解析，請用繁體中文回答，
				切記在使用MCP工具時一定要先詢問我並告知你預計傳入的參數"""
	)

	config = {
		"configurable": {
			"thread_id": "1"  
		}
	}

	async def main():
		while True:
			user_input = input()
			if user_input == "exit":
				return
			events = agent.astream(
				{"messages": user_input},
				config,
				stream_mode = "values",
			)
			output = []
			async for event in events:
				output.append(event["messages"][-1].pretty_repr(True))

			for line in output[-2:]:
				print(line)
			time.sleep(0.2)

	await main()

	state = await agent.aget_state(config)
	print(state)
	# print(state.values["messages"][-1].content)


================================ Human Message =================================

我叫甚麼名字
================================== Ai Message ==================================

您的名字是劉銓旭。
StateSnapshot(values={'messages': [HumanMessage(content='我叫甚麼名字', additional_kwargs={}, response_metadata={}, id='2a7145c2-9652-4b45-bb68-571e9e8daa15'), AIMessage(content='我不知道您的名字。', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'models/gemini-2.5-flash-preview-05-20', 'safety_ratings': []}, id='run--78f29183-bda5-47e5-bc1c-2e609810f6bb-0', usage_metadata={'input_tokens': 484, 'output_tokens': 5, 'total_tokens': 489, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='我叫劉銓旭', additional_kwargs={}, response_metadata={}, id='ae972d62-ea62-4a37-bcb1-5f8d5d6600bd'), AIMessage(content='好的，劉銓旭，我會記住您的名字。很高興認識您！還有什麼我能為您服務的嗎？', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 

In [10]:
# from pymongo import AsyncMongoClient
# async def main():
#     client = AsyncMongoClient(Mongo_url)
#     try:
#         await client.drop_database("checkpointing_db")
#     except Exception as e:
#         raise Exception("Unable to find the document due to the following error: ", e)
# await main()